<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7e14c0c5b6090b3957eb7bd1233448d60bc3921daeacb33c0a2c1e25d8538d61
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 16:06:37
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 94.89 K (0.68%)
Current PnL: -21.06 L (-14.03%)
CY Booked + Current PnL: -9.16 L (-6.1%)
-------------------
Total profit:  1.83 L
Total loss:  -22.89 L
-------------------
Total Booked + Current PnL: 17.82 L (14.47%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.47%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.25 L (62.84%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.44,59.0,X-LC,3.79,215008.0,10508.0,12793.0,0.69,5.14,5.95,11.39,10.0,0.82,1.54,14.55,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,98.93,55.0,M-SC,1.60,85687.0,-15090.0,15184.0,1.66,-14.97,17.72,0.09,245.0,-0.99,0.61,12.57,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.16,58.0,X-MC,1.27,212340.0,20598.0,17157.0,0.25,10.74,8.08,19.69,99.0,1.20,1.52,18.27,XY25,NTT,AC
33,ICICIGI,2260.25,-13.61,66.0,X-MC,2.07,148747.0,12614.0,18504.0,-0.51,9.27,12.44,22.86,91.0,0.68,1.07,23.05,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-12.91,57.0,X-LC,2.88,218135.0,7769.0,21006.0,1.75,3.69,9.63,13.68,37.0,0.37,1.57,20.83,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-45.08,51.0,M-SC,4.23,92835.0,1336.0,77415.0,4.63,1.46,83.39,86.07,223.0,0.02,0.67,34.81,XR,NTT,DURABLES
49,NESTLEIND,1377.00,-2.88,72.0,X-LC,2.62,298724.0,33298.0,23480.0,4.52,12.55,7.86,21.40,11.0,1.42,2.14,20.02,XY25,NTT,FMCG
66,SONACOMS,1006.00,-34.18,69.0,M-SC,12.29,81996.0,-19163.0,94049.0,3.60,-18.94,114.70,74.03,202.0,-0.20,0.59,15.62,AR,BTT,AUTO
82,VBL,671.64,-18.24,54.0,X-LC,10.58,294309.0,-21533.0,134205.0,3.58,-6.82,45.60,35.67,5.0,-0.16,2.11,6.09,X40N,ATH,FMCG
63,SHALBY,327.00,1188.68,56.0,M-SC,23.01,175156.0,-5987.0,51461.0,3.23,-3.31,29.38,25.10,235.0,-0.12,1.26,40.37,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,852.90,36.0,H-SC,5.21,82020.0,-26700.0,106380.0,-2.08,-24.56,129.70,73.29,161.0,-0.25,0.59,9.71,XR,NTT,ENTERTAINMENT
20,COFFEEDAY,80.00,-45.67,47.0,L-SC,28.72,80160.0,-33389.0,73443.0,-2.02,-29.40,91.62,35.27,268.0,-0.45,0.58,96.19,XR,NTT,HOTELS
81,VALIANTORG,838.00,-280.91,30.0,H-SC,4.20,100453.0,-63152.0,184472.0,-1.96,-38.60,183.64,74.15,139.0,-0.34,0.72,28.76,XR,NTT,CHEMICALS
53,RAJOOENG,143.10,-43.74,38.0,H-SC,17.78,85270.0,-17230.0,57830.0,-1.57,-16.81,67.82,39.61,114.0,-0.30,0.61,4.14,AR,ATH,MISC
50,PGHH,17907.65,-26.97,55.0,X-MC,3.24,211260.0,10440.0,57357.0,-1.36,5.20,27.15,33.76,80.0,0.18,1.52,10.67,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,70.0,M-SC,17.64,150201.0,981.0,143202.0,1.16,0.66,95.34,96.62,208.0,0.01,1.08,62.55,XY25,NTT,FINANCE
59,SAIL,228.0,43.54,50.0,M-MC,12.22,226236.0,1274.0,166374.0,0.84,0.57,73.54,74.53,192.0,0.01,1.62,33.00,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,62.0,H-SC,11.94,132273.0,-10206.0,29722.0,0.86,-7.16,22.47,13.70,163.0,-0.34,0.95,56.12,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.93,55.0,M-SC,1.60,85687.0,-15090.0,15184.0,1.66,-14.97,17.72,0.09,245.0,-0.99,0.61,12.57,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.90,220644.0,-49023.0,85345.0,-0.20,-18.18,38.68,13.47,138.0,-0.57,1.58,12.19,XY24,NTT,PAINTS
17,CERA,9475.0,-23.87,40.0,H-SC,2.21,138897.0,-37006.0,79032.0,-0.23,-21.04,56.90,23.89,149.0,-0.47,1.00,19.96,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.05,46.0,H-MC,7.14,102666.0,-29169.0,71394.0,0.67,-22.13,69.54,32.03,98.0,-0.41,0.74,15.00,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.14,207625.0,4455.0,37684.0,0.45,2.19,18.15,20.74,55.0,0.12,1.49,7.86,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,50.0,M-MC,12.22,226236.0,1274.0,166374.0,0.84,0.57,73.54,74.53,192.0,0.01,1.62,33.00,XY24,BTT,STEEL
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220414.0,15338.0,97225.0,-1.27,7.48,44.11,54.89,52.0,0.16,1.58,33.38,X5K,ATH,METALS
85,WIPRO,420.00,-10.62,61.0,M-LC,5.77,157362.0,6417.0,103041.0,1.44,4.25,65.48,72.51,53.0,0.06,1.13,10.53,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,55.0,M-LC,8.73,157827.0,16987.0,27509.0,-0.45,12.06,17.43,31.59,66.0,0.62,1.13,41.74,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220414.0,15338.0,97225.0,-1.27,7.48,44.11,54.89,52.0,0.16,1.58,33.38,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,13.00,192896.0,13088.0,99322.0,-0.73,7.28,51.49,62.52,88.0,0.13,1.38,43.85,XR,NTT,BANKS
85,WIPRO,420.00,-10.62,61.0,M-LC,5.77,157362.0,6417.0,103041.0,1.44,4.25,65.48,72.51,53.0,0.06,1.13,10.53,XR,NTT,IT
84,WHIRLPOOL,2270.00,-45.08,51.0,M-SC,4.23,92835.0,1336.0,77415.0,4.63,1.46,83.39,86.07,223.0,0.02,0.67,34.81,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.20,40.0,H-SC,8.07,124879.0,1543.0,130086.0,1.14,1.25,104.17,106.72,119.0,0.01,0.90,24.92,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144038.0,-132485.0,242560.0,1.52,-47.91,168.40,39.81,54.0,-0.55,1.03,1.52,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52199.0,-12807.0,163738.0,0.02,-19.70,313.68,232.18,198.0,-0.08,0.37,1.79,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.17,39.0,X-MC,1.82,230225.0,-17506.0,92573.0,0.79,-7.07,40.21,30.30,92.0,-0.19,1.65,5.29,X40,ATH,ELECTRICAL
34,ICICIPRULI,790.00,-22.94,40.0,X-MC,2.20,132662.0,-3126.0,45875.0,-0.17,-2.30,34.58,31.48,107.0,-0.07,0.95,9.54,X40,ATH,INSURANCE
12,BATAINDIA,2096.00,-36.29,42.0,X-SC,4.33,95259.0,-33411.0,78713.0,2.14,-25.97,82.63,35.20,219.0,-0.42,0.68,9.34,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,45.0,H-LC,0.62,155950.0,-30145.0,77632.0,0.01,-16.20,49.78,25.51,15.0,-0.39,1.12,14.92,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.16,58.0,X-MC,1.27,212340.0,20598.0,17157.0,0.25,10.74,8.08,19.69,99.0,1.20,1.52,18.27,XY25,NTT,AC
28,HAVELLS,2069.16,-7.17,39.0,X-MC,1.82,230225.0,-17506.0,92573.0,0.79,-7.07,40.21,30.30,92.0,-0.19,1.65,5.29,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.61,66.0,X-MC,2.07,148747.0,12614.0,18504.0,-0.51,9.27,12.44,22.86,91.0,0.68,1.07,23.05,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-2.88,72.0,X-LC,2.62,298724.0,33298.0,23480.0,4.52,12.55,7.86,21.40,11.0,1.42,2.14,20.02,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144038.0,-132485.0,242560.0,1.52,-47.91,168.40,39.81,54.0,-0.55,1.03,1.52,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52199.0,-12807.0,163738.0,0.02,-19.70,313.68,232.18,198.0,-0.08,0.37,1.79,XY24,NTT,MISC
73,TCS,4389.97,-29.97,45.0,X-LC,12.48,279246.0,-66710.0,133424.0,0.03,-19.28,47.78,19.28,1.0,-0.50,2.00,3.23,X40,ATH,IT
9,BAJAJHFL,181.50,-14.27,45.0,X-MC,5.81,180466.0,-20040.0,117195.0,-0.08,-9.99,64.94,48.45,90.0,-0.17,1.30,4.07,X40N,ATH,FINANCE
2,ACC,3906.00,-37.64,52.0,X-MC,3.36,186010.0,-51941.0,204592.0,0.15,-21.83,109.99,64.15,174.0,-0.25,1.33,4.34,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52199.0,-12807.0,163738.0,0.02,-19.70,313.68,232.18,198.0,-0.08,0.37,1.79,XY24,NTT,MISC
55,RELAXO,1176.00,-39.20,45.0,X-SC,3.99,82946.0,-62214.0,138147.0,-0.18,-42.86,166.55,52.31,136.0,-0.45,0.60,11.03,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.29,42.0,X-SC,4.33,95259.0,-33411.0,78713.0,2.14,-25.97,82.63,35.20,219.0,-0.42,0.68,9.34,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.61,48.0,X-SC,1.02,107262.0,5258.0,38668.0,-0.14,5.15,36.05,43.06,122.0,0.14,0.77,23.15,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,44.0,X-SC,2.40,108270.0,-19668.0,66803.0,-0.18,-15.37,61.70,36.84,143.0,-0.29,0.78,10.84,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.48,279246.0,-66710.0,133424.0,0.03,-19.28,47.78,19.28,1.0,-0.50,2.00,3.23,X40,ATH,IT
39,INFY,2275.00,-19.60,46.0,X-LC,7.54,313430.0,542.0,171133.0,-0.20,0.17,54.60,54.87,3.0,0.00,2.25,6.83,X40,BTT,IT
41,ITC,452.00,-38.77,51.0,X-LC,2.88,198118.0,-2020.0,22902.0,1.31,-1.01,11.56,10.44,4.0,-0.09,1.42,4.50,X40,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.58,294309.0,-21533.0,134205.0,3.58,-6.82,45.60,35.67,5.0,-0.16,2.11,6.09,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,51.0,H-LC,7.24,250176.0,-11443.0,130667.0,0.74,-4.37,52.23,45.57,7.0,-0.09,1.80,6.45,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6750.00,57.0,L-SC,6.17,75150.0,-18660.0,95275.0,2.06,-19.89,126.78,81.67,269.0,-0.20,0.54,45.53,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,47.0,L-SC,28.72,80160.0,-33389.0,73443.0,-2.02,-29.40,91.62,35.27,268.0,-0.45,0.58,96.19,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,45.0,H-SC,9.48,90855.0,-7125.0,28728.0,0.58,-7.27,31.62,22.05,152.0,-0.25,0.65,32.06,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,62.0,H-SC,11.94,132273.0,-10206.0,29722.0,0.86,-7.16,22.47,13.70,163.0,-0.34,0.95,56.12,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,52.0,H-SC,10.25,143891.0,-35180.0,100911.0,-0.70,-19.65,70.13,36.71,135.0,-0.35,1.03,42.77,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,70.0,M-SC,17.64,150201.0,981.0,143202.0,1.16,0.66,95.34,96.62,208.0,0.01,1.08,62.55,XY25,NTT,FINANCE
4,ANGELONE,3033.00,14.80,69.0,X-SC,9.38,203573.0,12567.0,45132.0,1.53,6.58,22.17,30.21,157.0,0.28,1.46,28.83,X40N,NTT,FINANCE
68,SURYODAY,240.00,57.89,52.0,H-SC,10.25,143891.0,-35180.0,100911.0,-0.70,-19.65,70.13,36.71,135.0,-0.35,1.03,42.77,XR,NTT,BANKS
63,SHALBY,327.00,1188.68,56.0,M-SC,23.01,175156.0,-5987.0,51461.0,3.23,-3.31,29.38,25.10,235.0,-0.12,1.26,40.37,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220414.0,15338.0,97225.0,-1.27,7.48,44.11,54.89,52.0,0.16,1.58,33.38,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.09
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.31
LC    33.84
MC    21.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.55
X40      14.77
XY25     12.31
XR       10.93
X40N     10.24
AR        8.17
OX40N     7.63
X200      1.82
X5K       1.58
SR        1.13
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.42
X-LC    22.74
X-MC    16.09
M-SC    12.77
M-LC     5.33
H-LC     4.74
X-SC     4.66
H-MC     3.80
M-MC     1.62
L-SC     1.46
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.23,-3.44,37.53
IT,12.93,-16.66,79.75
FINANCE,12.22,-11.07,61.42
MISC,6.85,-20.29,86.94
BANKS,6.34,-12.70,71.98
PAINTS,5.57,-18.93,36.66
ELECTRICAL,5.35,-10.58,49.38
INSURANCE,3.93,-0.88,38.59
AUTO,2.85,-44.74,113.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3311670.0,22
XR,1386693.0,14
AR,1191413.0,9
X40,779717.0,10
X40N,601940.0,8
OX40N,583361.0,10
XY25,467556.0,8
SR,264391.0,2
X5K,97225.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3455160.0,25
M-SC,1262617.0,15
X-MC,1083581.0,11
X-LC,1070193.0,13
X-SC,531201.0,6
H-MC,350049.0,3
H-LC,280635.0,3
M-LC,265459.0,4
L-SC,258106.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197484.0      6
           XR         828497.0      7
           AR         822011.0      5
M-SC       XY24       683478.0      6
X-MC       XY24       586777.0      4
X-LC       X40        486695.0      5
           XY24       306977.0      2
X-SC       X40N       288750.0      4
H-SC       OX40N      274410.0      4
           SR         264391.0      2
X-MC       X40        214309.0      4
H-LC       AR         208299.0      2
X-LC       X40N       195995.0      3
H-MC       XY24       179333.0      1
L-SC       XR         168718.0      2
M-MC       XY24       166374.0      1
X-MC       XY25       165300.0      2
X-SC       XY24       163738.0      1
M-SC       AR         161103.0      2
           OX40N      148169.0      4
           XY25       143202.0      1
           XR         126665.0      2
X-MC       X40N       117195.0      1
M-LC       XR         103041.0      1
H-MC       XR          99322.0      1
M-LC       X5K         97225.0      1
L-SC       OX40N       89388.0      1
X-LC       XY25        80526.0      3
X-SC       X40         78713.0      1
H-LC       X200        72336.0      1
H-MC       OX40N       71394.0      1
H-SC       MH          68367.0      1
L-MC       XR          60450.0      1
L-LC       XY25        40844.0      1
M-LC       XY25        37684.0      1
           XY24        27509.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
